# Anomaly_Detection

In [15]:
!pip install pycaret
!pip install sklearn
!pip install pandas

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [17]:
# loading sample dataset from pycaret dataset module
from pycaret.datasets import get_data
data = get_data('anomaly')

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,0.263995,0.764929,0.138424,0.935242,0.605867,0.518790,0.912225,0.608234,0.723782,0.733591
1,0.546092,0.653975,0.065575,0.227772,0.845269,0.837066,0.272379,0.331679,0.429297,0.367422
2,0.336714,0.538842,0.192801,0.553563,0.074515,0.332993,0.365792,0.861309,0.899017,0.088600
3,0.092108,0.995017,0.014465,0.176371,0.241530,0.514724,0.562208,0.158963,0.073715,0.208463
4,0.325261,0.805968,0.957033,0.331665,0.307923,0.355315,0.501899,0.558449,0.885169,0.182754


In [18]:
# import pycaret anomaly and init setup
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [20]:
from pycaret.anomaly import AnomalyExperiment
exp = AnomalyExperiment()

In [21]:
# init setup on exp
exp.setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [22]:
available_models=models()
available_models

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pycaret.internal.patches.pyod.CBLOFForceToDouble
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [23]:
model_ids = list(available_models.index)
model_ids

['abod',
 'cluster',
 'cof',
 'iforest',
 'histogram',
 'knn',
 'lof',
 'svm',
 'pca',
 'mcd',
 'sod',
 'sos']

# Anomaly Count with its parameters

In [38]:
import pandas as pd
anomaly_details = []

for m in model_ids[:-2]:
    model = exp.create_model(m)
    anomalies = exp.assign_model(model)

    if not isinstance(data.columns, pd.RangeIndex):
        data = data.reset_index(drop=True)

    pred = exp.predict_model(model, data=data)
    anomaly_count = pred[pred['Anomaly'] == 1].shape[0]

    try:
        params = model.get_params()
    except AttributeError:
        params = {}

    parameter_columns = {}
    for i, (param_name, param_value) in enumerate(params.items(), start=1):
        parameter_columns[f"Parameter {i}"] = f"{param_name}={param_value}"
    entry = {"Model": m, "Anomaly_Count": anomaly_count, **parameter_columns}
    anomaly_details.append(entry)

df_anomalies = pd.DataFrame(anomaly_details)

df_anomalies = df_anomalies.fillna("")
df_anomalies.to_csv("anomaly_details.csv", index=False)

display(df_anomalies.head(10))



Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

,Model,Anomaly_Count,Parameter 1,Parameter 2,Parameter 3,Parameter 4,Parameter 5,Parameter 6,Parameter 7,Parameter 8,Parameter 9,Parameter 10,Parameter 11
0,abod,0,contamination=0.05,method=fast,n_neighbors=5,,,,,,,,
1,cluster,50,alpha=0.9,beta=5,check_estimator=False,clustering_estimator=None,contamination=0.05,n_clusters=8,n_jobs=None,random_state=123,use_weights=False,,
2,cof,50,contamination=0.05,method=fast,n_neighbors=20,,,,,,,,
3,iforest,50,behaviour=new,bootstrap=False,contamination=0.05,max_features=1.0,max_samples=auto,n_estimators=100,n_jobs=-1,random_state=123,verbose=0,,
4,histogram,50,alpha=0.1,contamination=0.05,n_bins=10,tol=0.5,,,,,,,
5,knn,46,algorithm=auto,contamination=0.05,leaf_size=30,method=largest,metric=minkowski,metric_params=None,n_jobs=-1,n_neighbors=5,p=2,radius=1.0,
6,lof,46,algorithm=auto,contamination=0.05,leaf_size=30,metric=minkowski,metric_params=None,n_jobs=-1,n_neighbors=20,novelty=True,p=2,,
7,svm,50,cache_size=200,coef0=0.0,contamination=0.05,degree=3,gamma=auto,kernel=rbf,max_iter=-1,nu=0.5,shrinking=True,tol=0.001,verbose=False
8,pca,50,contamination=0.05,copy=True,iterated_power=auto,n_components=None,n_selected_components=None,random_state=123,standardization=True,svd_solver=auto,tol=0.0,weighted=True,whiten=False
9,mcd,50,assume_centered=False,contamination=0.05,random_state=123,store_precision=True,support_fraction=None,,,,,,


# silhouette_score

In [39]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

result = []

for m in model_ids[:-2]:
    model = exp.create_model(m)
    anomalies = exp.assign_model(model)

    if not isinstance(data.columns, pd.RangeIndex):
        data = data.reset_index(drop=True)

    pred = exp.predict_model(model, data=data)
    anomaly_count = pred[pred['Anomaly'] == 1].shape[0]
    filtered_data = pred[pred['Anomaly'] == 0]
    X = filtered_data.drop(columns=['Anomaly', 'Anomaly_Score'], errors='ignore')
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    best_kmeans_score, best_meanshift_score, best_hierarchical_score = 0, 0, 0
    best_kmeans_clusters, best_hierarchical_clusters = 0, 0
    meanshift_clusters = None

    # K-Means
    for i in range(2, 11):
        kmeans = KMeans(n_clusters=i, random_state=123)
        cluster_labels = kmeans.fit_predict(X_scaled)
        silhouette = silhouette_score(X_scaled, cluster_labels)
        if silhouette > best_kmeans_score:
            best_kmeans_score = silhouette
            best_kmeans_clusters = i

    # Hierarchical Clustering
    for i in range(2, 11):
        hierarchical = AgglomerativeClustering(n_clusters=i)
        cluster_labels = hierarchical.fit_predict(X_scaled)
        silhouette = silhouette_score(X_scaled, cluster_labels)
        if silhouette > best_hierarchical_score:
            best_hierarchical_score = silhouette
            best_hierarchical_clusters = i

    result.append({
        "Model": m,
        "Anomaly_Count": anomaly_count,
        "KMeans_Clusters": best_kmeans_clusters,
        "KMeans_Silhouette": best_kmeans_score,
        "Hierarchical_Clusters": best_hierarchical_clusters,
        "Hierarchical_Silhouette": best_hierarchical_score
    })

df = pd.DataFrame(result)
df.to_csv("model_metrics_with_clustering.csv", index=False)
display(df.head(10))


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

,Model,Anomaly_Count,KMeans_Clusters,KMeans_Silhouette,Hierarchical_Clusters,Hierarchical_Silhouette
0,abod,0,2,0.209659,2,0.202672
1,cluster,50,2,0.225795,2,0.223187
2,cof,50,2,0.224842,2,0.222678
3,iforest,50,3,0.212808,2,0.218060
4,histogram,50,3,0.214977,2,0.217355
5,knn,46,2,0.224637,2,0.222071
6,lof,46,2,0.224754,2,0.222126
7,svm,50,3,0.208759,2,0.211764
8,pca,50,3,0.209207,2,0.209719
9,mcd,50,2,0.224968,2,0.214018
